## Import Libraries

In [358]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score


## 1A. 


In [359]:
boston_df = pd.read_csv("BostonHousing.csv")
print("Number of Rows ",boston_df.shape[0])
print("Number of Columns: ",boston_df.shape[1],"\n"*2)
print("First 5 records of Data are: ")
boston_df.head()
#I still need to extract space between column names

Number of Rows  506
Number of Columns:  14 


First 5 records of Data are: 


,CRIME,ZONE,INDUST,CHAR RIV,NIT OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST RATIO,LOW STAT,MVALUE,C MVALUE
0,0.00632,18.0,2.31,N,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,No
1,0.02731,0.0,7.07,N,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,No
2,0.02729,0.0,7.07,N,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,Yes
3,0.03237,0.0,2.18,N,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,Yes
4,0.06905,0.0,2.18,N,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,Yes


## 1B

In [360]:
# We will strip trailing spaces and replace the remaining spaces with an underscore _. Instead of using 
# the `rename` method, we  create a modified copy of `columns` and assign to the `columns` field of the dataframe.

print('Modified column titles with no space and one word for titles:',"\n")
boston_df.columns = [s.strip().replace(' ', '_') for s in boston_df.columns]
print(boston_df.columns)


Modified column titles with no space and one word for titles: 

Index(['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE',
       'C_MVALUE'],
      dtype='object')


## 1C

In [361]:
print(boston_df.dtypes)

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object


In [362]:
# WE Can see only two columns have object data type:
print("Column with Object data type in the dataset are: ")
for i in boston_df.columns:
    if boston_df[i].dtype == "O":
        print(i)
            

# Lets change these variable type to 'category'.
boston_df.CHAR_RIV = boston_df.CHAR_RIV.astype('category')
boston_df.C_MVALUE = boston_df.C_MVALUE.astype('category')


# Display category levels (attributes) and category type FOR CHAR_RIV
print("")
print('Category levels and changed variable type of CHAR_RIV:')
print(boston_df.CHAR_RIV.cat.categories) 
print(boston_df.CHAR_RIV.dtype)


# Display category levels (attributes) and category type FOR C_MVALUE
print("")
print('Category levels and changed variable type of C_MVALUE:')
print(boston_df.C_MVALUE.cat.categories) 
print(boston_df.C_MVALUE.dtype)


Column with Object data type in the dataset are: 
CHAR_RIV
C_MVALUE

Category levels and changed variable type of CHAR_RIV:
Index(['N', 'Y'], dtype='object')
category

Category levels and changed variable type of C_MVALUE:
Index(['No', 'Yes'], dtype='object')
category


In [363]:
# Lets convert these categorical column to Dummy vraibales
boston_df = pd.get_dummies(boston_df, prefix_sep='_', drop_first=True)
boston_df.columns

Index(['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE', 'CHAR_RIV_Y',
       'C_MVALUE_Yes'],
      dtype='object')

## 1D

In [364]:
boston_df.describe()

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806,0.069170,0.166008
std,8.601545,23.322453,6.860353,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104,0.253994,0.372456
min,0.006320,0.000000,0.460000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000,0.000000,0.000000
25%,0.082045,0.000000,5.190000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000,0.000000,0.000000
50%,0.256510,0.000000,9.690000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000,0.000000,0.000000
75%,3.677083,12.500000,18.100000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000,0.000000,0.000000
max,88.976200,100.000000,27.740000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000,1.000000,1.000000


We know that in case of N missing values in a column, count for the same column will be reduced by N. In boston_df data frame, count for each column is equal to the total number of rows of the data frame. This helps us to conclude that there is no missing value in any of the column in the entire data frame

____________________________________________________________________________________________________________________

## 2A

In [365]:
#Partitioning Data

# Identify predictors(col names) and outcome(target Name) of the regression model]\
# These are just names ( not data)
predictors = ['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']

outcome = 'MVALUE'



# Identify X and y variables for regression and partition data
# using 60% of records for training and 40% for validation (test_size=0.4). 
X = boston_df[predictors]
Y = boston_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, Y, test_size=0.4, random_state=1)

In [366]:
#Creating Linear Regression Model
#We have already imported LinearRegression method from Skilearn (skikit-learn) and we will use it now
lm = LinearRegression()
lm.fit(train_X, train_y)

#Coffeienct of line for entire Model
print(lm.intercept_)
#Coffient of each of line w.r.t Simple Linear Rgesression
print(lm.coef_)


# Presenting out put in tabular form:
print("")
print("")
print('Regression Model for BostonHousing Training Set:')
print('Intercept: ', np.round(lm.intercept_,2))
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(lm.coef_,2)}))

43.65353695296477
[-1.43638902e-01  8.08022501e-03  1.16465806e-01 -1.64685693e+01
  8.93409024e-01 -5.28222118e-03 -7.19404813e-01  1.99053114e-01
 -7.29552121e-03 -5.81312293e-01 -4.51815711e-01  2.10943383e+00
  1.09908492e+01]


Regression Model for BostonHousing Training Set:
Intercept:  43.65
       Predictor  Coefficient
0          CRIME        -0.14
1           ZONE         0.01
2         INDUST         0.12
3      NIT_OXIDE       -16.47
4          ROOMS         0.89
5            AGE        -0.01
6       DISTANCE        -0.72
7         RADIAL         0.20
8            TAX        -0.01
9       ST_RATIO        -0.58
10      LOW_STAT        -0.45
11    CHAR_RIV_Y         2.11
12  C_MVALUE_Yes        10.99


MVALUE = 43.65 -0.14(CRIME) +0.01(ZONE) +0.12(INDUST) - 16.47(NIT_OXIDE) + 0.89(ROOMS) -0.01(AGE) -0.72(DISTANCE) + 0.20(RADIAL) -0.01(TAX) -0.58(ST_RATIO) -0.45(LOW_STAT) +2.11(CHAR_RIV_Y) + 10.99(C_MVALUE_Yes)

## 2B

In [367]:
# Use predict() function to make predictions for training set and also for validation set as well
#Prediction for training set
pred_y = lm.predict(train_X)

#Model Prediction for Validation set
lm_pred = lm.predict(valid_X)

#Lets Calcuate the R square
r2 = round(r2_score(train_y, pred_y),3)
adj_r2 = round(adjusted_r2_score(train_y, pred_y, lm),3)
aic = round(AIC_score(train_y, pred_y, lm),2)
bic = round(BIC_score(train_y, pred_y, lm),2)


# Display prediction performance measures for training set.
print('Prediction Performance Measures for Training Set')
print('r2 : ', r2)
print('Adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)
print() 

# Create prediction performance measures for validation set.
r2 = round(r2_score(valid_y, lm_pred),3)
adj_r2 = round(adjusted_r2_score(valid_y, lm_pred, lm),3)
aic = round(AIC_score(valid_y, lm_pred, lm),2)
bic = round(BIC_score(valid_y, lm_pred, lm),2)

# Display prediction performance measures for validation set.
print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2)
print('adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)

Prediction Performance Measures for Training Set
r2 :  0.839
Adjusted r2 :  0.832
AIC :  1663.52
BIC :  1719.22

Prediction Performance Measures for Validation Set
r2 :  0.834
adjusted r2 :  0.822
AIC :  1156.17
BIC :  1205.87


- We can see R square and adjusted R square are very close. Hence there is no overfitting in our model

## 2C

In [368]:
#Another info to extract from the Model is What VALUE have i predicted, actual price and Residual (Difference)
print('Actual, Prediction, and Residual Prices for Validation Set')
result = round(pd.DataFrame({'Actual': valid_y,'Predicted': lm_pred, 
                       'Residual': valid_y - lm_pred}), 2)
print(result.head(10))

Actual, Prediction, and Residual Prices for Validation Set
     Actual  Predicted  Residual
307    28.2      25.53      2.67
343    23.9      22.95      0.95
47     16.6      17.89     -1.29
67     22.0      21.81      0.19
362    20.8      18.89      1.91
132    23.0      19.60      3.40
292    27.9      25.95      1.95
31     14.5      17.90     -3.40
218    21.5      22.40     -0.90
90     22.6      23.25     -0.65


In [369]:
#But instead of observing difference for each values of each row, we can directly extract the error summary

# Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, lm_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.5845
            Mean Absolute Error (MAE) : 2.5961
          Mean Percentage Error (MPE) : -2.7127
Mean Absolute Percentage Error (MAPE) : 13.1715

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.4347
       Root Mean Squared Error (RMSE) : 3.8763
            Mean Absolute Error (MAE) : 2.7696
          Mean Percentage Error (MPE) : -2.2773
Mean Absolute Percentage Error (MAPE) : 13.3233


_________________________________________________________________________________________________________________

## 3A

In [370]:
#Exhautive serach Method:

#It takes three argumnets (The X column names used for modeling, The Model itself, and adjusted R square)
#Then it provides reslut of adjusted R square for all variables 
#Note that instead of  adj R square in third argument, we can put any calculation we want to see


#Argument 1: All X columns
allVariables = train_X.columns

# Arugments 2: Now lets Define train_model() function used in Executive Search algorithm.
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Argument 3: Adj R Square. Define score_model() function used in Executive Search
# algorithm with executive_search() function. 
def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # Apply negative sign as score is optimized to be 
    # as low as possible in exhaustive_search() function.
    return -adjusted_r2_score(train_y, pred_y, model)



# Now Lets launch exhaustive_search function with three arguments
results = exhaustive_search(allVariables, train_model, score_model)


#The results of above step can be directly seen but we will modify it to present in a better way 
data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)


#In Python We can also see the output within our own witdh dispay on screen
pd.set_option('display.width', 100)


# Display the Exhaustive Search results.
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))


# Reset the output width to the default. 
pd.reset_option('display.width')

     n     r2adj          AIC    AGE  CHAR_RIV_Y  CRIME  C_MVALUE_Yes  DISTANCE  INDUST  LOW_STAT  \
0    1  0.604171  1911.931006  False       False  False          True     False   False     False   
1    2  0.793030  1716.454818  False       False  False          True     False   False      True   
2    3  0.804621  1699.980252  False       False   True          True     False   False      True   
3    4  0.810106  1692.338134  False        True   True          True     False   False      True   
4    5  0.814675  1685.940421  False        True   True          True     False   False      True   
5    6  0.821125  1676.183684  False       False   True          True      True   False      True   
6    7  0.826096  1668.619824  False       False   True          True      True   False      True   
7    8  0.830529  1661.766431  False        True   True          True      True   False      True   
8    9  0.831819  1660.418877  False        True   True          True      True   False    

We can from the output that 11 Predictors ( excluding AGE and Zone), our model has best adjusted rSquare of 0.833103

In [371]:
# Develop the multiple linear regression model based on the Exhaustive Search results.

# Identify predictors and outcome of the regression model.
predictors_ex = ['CRIME', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']

outcome = 'MVALUE'

#Identify X and y variables for regression and partition data using 60% of records for training and 40% for validation

X = boston_df[predictors_ex]
y = boston_df[outcome]
train_X_ex, valid_X_ex, train_y_ex, valid_y_ex = \
          train_test_split(X, y, test_size=0.4, random_state=1)

# Create multiple linear regression model using X and y.
lm_ex = LinearRegression()
lm_ex.fit(train_X_ex, train_y_ex)

# Display intercept and regression coefficients. Round them to 2 decimals.
print('Regression Model for Training Set Using Exhaustive Search')
print()
print('Intercept ', np.round(lm_ex.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(lm_ex.coef_, 2)}))

Regression Model for Training Set Using Exhaustive Search

Intercept  43.89
       Predictor  Coefficient
0          CRIME        -0.14
1         INDUST         0.11
2      NIT_OXIDE       -16.89
3          ROOMS         0.86
4       DISTANCE        -0.63
5         RADIAL         0.19
6            TAX        -0.01
7       ST_RATIO        -0.61
8       LOW_STAT        -0.46
9     CHAR_RIV_Y         2.13
10  C_MVALUE_Yes        11.11


MVALUE = 43.89 -0.14(CRIME)  +0.11(INDUST) - 16.89(NIT_OXIDE) + 0.86(ROOMS) -0.63(DISTANCE) + 0.19(RADIAL) -0.01(TAX) -0.61(ST_RATIO) -0.46(LOW_STAT) +2.13(CHAR_RIV_Y) + 11.11(C_MVALUE_Yes)

In [372]:
# Use predict() function make predictions for validation set and measure their accuracy.
lm_ex_pred = lm_ex.predict(valid_X_ex)



# Develop and display data frame with actual values of Price, scoring (predicted) results, and residuals.
result = round(pd.DataFrame({'Actual': valid_y_ex,'Predicted': lm_ex_pred, 
                       'Residual': valid_y_ex - lm_ex_pred}), 2)
print()
print('Prediction for Validation Set Using Exhaustive Search') 
print(result.head(10))

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Exhaustive Search')
regressionSummary(valid_y_ex, lm_ex_pred)


Prediction for Validation Set Using Exhaustive Search
     Actual  Predicted  Residual
307    28.2      25.24      2.96
343    23.9      22.78      1.12
47     16.6      18.17     -1.57
67     22.0      21.86      0.14
362    20.8      18.93      1.87
132    23.0      19.58      3.42
292    27.9      25.25      2.65
31     14.5      18.06     -3.56
218    21.5      22.49     -0.99
90     22.6      23.28     -0.68

Accuracy Measures for Validation Set Using Exhaustive Search

Regression statistics

                      Mean Error (ME) : 0.4505
       Root Mean Squared Error (RMSE) : 3.8674
            Mean Absolute Error (MAE) : 2.7724
          Mean Percentage Error (MPE) : -2.1963
Mean Absolute Percentage Error (MAPE) : 13.3441


- We can see with optimization of selection of predictor our RMSE has decreased slightly from 3.87 to 3.86

## 3B

In [373]:
# Define train_model() function used in Forward Selection
# algorithm with forward_selection() function. 
# The initial model is the constant model - this requires 
# special handling in train_model and score_model.

def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Forward Selection
# algorithm with forward_selection() function. 
def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

# Use forward_selection() function to identify the
# best_model and best_variables.
best_model_fs, best_variables_fs = forward_selection(train_X.columns, 
                    train_model, score_model, verbose=True)

# Display best variables based on Forward Selection algorithm.
print()
print('Best Variables from Forward Selection Algorithm')
print(best_variables_fs)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=2191.75, constant
Step: score=1911.93, add C_MVALUE_Yes
Step: score=1716.45, add LOW_STAT
Step: score=1699.98, add CRIME
Step: score=1692.34, add CHAR_RIV_Y
Step: score=1685.94, add ST_RATIO
Step: score=1682.90, add ROOMS
Step: score=1680.20, add DISTANCE
Step: score=1665.78, add NIT_OXIDE
Step: score=1660.42, add RADIAL
Step: score=1660.42, add None

Best Variables from Forward Selection Algorithm
['C_MVALUE_Yes', 'LOW_STAT', 'CRIME', 'CHAR_RIV_Y', 'ST_RATIO', 'ROOMS', 'DISTANCE', 'NIT_OXIDE', 'RADIAL']


In [374]:
# Develop the multiple linear regression model based
# on the Forward Selection results.

# Identify predictors and outcome of the regression model.
predictors_fs = ['CRIME', 'NIT_OXIDE', 'ROOMS', 'DISTANCE',
       'RADIAL', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']

outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 60% of records for training and 40% for validation 
# (test_size=0.4). 
X = boston_df[predictors_fs]
y = boston_df[outcome]
train_X_fs, valid_X_fs, train_y_fs, valid_y_fs = \
          train_test_split(X, y, test_size=0.4, random_state=1)

# Create multiple linear regression model using X and y.
lm_fs = LinearRegression()
lm_fs.fit(train_X_fs, train_y_fs)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Forward Selection')
print()
print('Intercept ', np.round(lm_fs.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(lm_fs.coef_, 2)}))

Regression Model for Training Set Using Forward Selection

Intercept  42.76
      Predictor  Coefficient
0         CRIME        -0.14
1     NIT_OXIDE       -15.95
2         ROOMS         0.87
3      DISTANCE        -0.71
4        RADIAL         0.11
5      ST_RATIO        -0.60
6      LOW_STAT        -0.45
7    CHAR_RIV_Y         2.36
8  C_MVALUE_Yes        10.97


MVALUE = 42.76 -0.14(CRIME) - 15.95(NIT_OXIDE) + 0.87(ROOMS) -0.71(DISTANCE) + 0.11(RADIAL) -0.60(ST_RATIO) -0.45(LOW_STAT) +2.36(CHAR_RIV_Y) + 10.97(C_MVALUE_Yes)

In [375]:
# Use predict() to score predictions for validation set.
lm_fs_pred = lm_fs.predict(valid_X_fs)

# Develop and display data frame with actual values of Price,
# scoring (predicted) results, and residuals.
# Use round() function to round vlaues in data frame to 
# 2 decimals. 
result = round(pd.DataFrame({'Actual': valid_y_fs,'Predicted': lm_fs_pred, 
                       'Residual': valid_y_fs - lm_fs_pred}), 2)
print()
print('Predictions for Validation Set Using Forward Selection')
print(result.head(10))

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Forward Selection')
regressionSummary(valid_y_fs, lm_fs_pred)


Predictions for Validation Set Using Forward Selection
     Actual  Predicted  Residual
307    28.2      25.20      3.00
343    23.9      23.48      0.42
47     16.6      17.85     -1.25
67     22.0      22.11     -0.11
362    20.8      18.97      1.83
132    23.0      19.27      3.73
292    27.9      25.07      2.83
31     14.5      18.21     -3.71
218    21.5      22.04     -0.54
90     22.6      23.86     -1.26

Accuracy Measures for Validation Set Using Forward Selection

Regression statistics

                      Mean Error (ME) : 0.4321
       Root Mean Squared Error (RMSE) : 3.9314
            Mean Absolute Error (MAE) : 2.8585
          Mean Percentage Error (MPE) : -2.3792
Mean Absolute Percentage Error (MAPE) : 13.8040


- We can see Common Accuracy Measure errors are high for predictors from Forward selcetion
- Hence we will stick to 11 Predictors resluted from exhautive search Method